In [4]:
import os
import pandas as pd
import numpy as np
from scipy.sparse import load_npz
import sys
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import math
from scipy.stats import norm
from healthrex_ml.evaluators import BinaryEvaluator
from tqdm import tqdm
from functions.trainers import LightGBMTrainer

In [5]:
trainer = LightGBMTrainer(working_dir="/deep/group/aihc/win23/EHR/20230218_jyx_mcp_artifacts")

In [6]:
tasks = ['label_NA', 'label_K', 'label_CR', 'label_CA', 'label_ALB']

In [7]:
delta = {
    "label_NA": 2.5,
    "label_K": 0.5,
    "label_CR": 0.125,
    "label_CA": 0.525,
    "label_ALB": 0.425
}

In [ ]:
for task in tasks:
    trainer(task, delta[task], True)

Training until validation scores don't improve for 100 rounds
